# Задание
В папке с заданием Вы найдете:


*   файл train.csv, содержащий обучающую выборку и соответствующие значения целевой переменной;
*   файл test.csv, содержащий тестовую выборку без соответствующих значений целевой переменной;
*   файлы users.csv, users_friends.csv, games_details.csv, achievements_stats.csv, содержающие дополнительные данные, которые могут помочь при решении задачи;
*   файл simple_pipeline.ipynb, содержащий подробное описание данных и пример получения столбца значений целевой переменной для открытой тестовой выборки;
*   файл sample_submission.csv, содержащий пример корректной посылки.

# Постановка задачи
Объектами изучения в этом задании будут пользователи онлайн-магазина компьютерных игр Steam и сами игры. Основная величина, которую требуется предсказать, -- это время, проведенное пользователем в той или иной игре. Пользователи при этом описываются датой регистрации, дружескими связями, достижениями и другими признаками, а про игры известна их жанровая принадлежность и разметка по некоторым другим тегам (подробнее -- в ноутбуке). Вам предлагается проверить гипотезу о зависимости времени, проведенного пользователем в игре, от свойств игры и статистики самого пользователя и разработать модель машинного обучения для восстановления этой зависимости.
# Evaluation
Оценка качества модели определяется с помощью вычисления rMSLE (root mean squared logarithmic error) -- корня из среднеквадратической ошибки.




# Описание решения

В основе решения лежит feature engineering

1. Объединить датасеты c train/test
2. Исключить столбец profilestate из-за его неинформативности (все значения равны 1).
3. Обучить модель на числовых данных с помощью LightGBM без дополнительной обработки датасета <font color="green">(Улучшение)</font>.
4. Создать категориальный признак, выделив топ N самых популярных игр, остальные обозначить как "Other" <font color="green">(Улучшение)</font>.
5. Выполнить one-hot кодирование тегов для топ N игр <font color="green">(Улучшение)</font>.
6. Применить one-hot кодирование для жанров игр <font color="green">(Улучшение)</font>.
7. Сагрегировать данные по достижениям:
    1. Определить достижение с наименьшим процентом выполнения среди игроков <font color="green">(Улучшение)</font>.
    2. Посчитать количество достижений <font color="red">(Ухудшение)</font>.
    3. Определить количество скрытых достижений <font color="green">(Улучшение)</font>.
8. Ввести временной признак с момента выхода игры и создания профиля в Steam <font color="green">(Улучшение)</font>.
9. Данные о друзьях:
    1. Число друзей <font color="green">(Улучшение)</font>.
    2. Поскольку большинство пользователей имеют не более 5 друзей в датасете, дальнейшая агрегация данных о них нецелесообразна.
10. Оптимизировать значение N для категорийных признаков (из пункта 4 и 5), выбирая параметры по популярности и стандартному отклонению, чтобы в тренировочной выборке было не менее 50 образцов <font color="blue">(не дало сильного прироста)</font>.
11. Число игр, в которые играл пользователь <font color="green">(Улучшение)</font>.
12. Обнаружено, что 99% пользователей в тестовой выборке известны в тренировочной, что позволяет агрегацию данных о пользователях, включая все игры, в которые они играли.
13. Агрегировать среднее количество часов, проведенных в онлайн и оффлайн играх, с осторожностью тестировать <font color="orange">(агрегация признака исключительно на обучающей выборке, чтобы избежать утечки данных)</font> и тестировать аналогичные признаки:
    1. Агрегировать признаки на части обучающей выборки <font color="green">(Улучшение)</font>.
    2. Среднее время для каждой игры <font color="green">(Улучшение)</font>.
    3. Вычислить уровень увлеченности пользователя:
    <font color="green">(Улучшение)</font>
$$
Увлеченность(N, M) = \text{(Число минут для пользователя N в игре M)} - \text{(Среднее число минут для игры M)}
$$
Среднее время по игре рассчитывается без учета нулевых значений. Уровень увлеченности пользователя определяется как:
$$
Увлеченность(N) = \frac{1}{|games(N)|} \sum_{M \in games(N)} Увлеченность(N, M)
$$
    4. Определить признак $\text{среднее\_время\_игры(M) + Увлеченность(N)}$ <font color="green">(Улучшение)</font>.
14. Эксперименты с графом друзей с использованием networkx:
    1. Количество "команд" для каждой игры (число компонент связности подграфа игры) <font color="green">(Улучшение)</font>.
    2. Расстояние до самого дальнего человека в "команде" (эксцентриситет вершины в подграфе игры) <font color="green">(Улучшение)</font>.
    3. Число самой сообщенной команды (максимальная клика в подграфе игры) <font color="red">(Ухудшение)</font>.
    4. Комбинация признаков из пунктов 3 и 4 <font color="red">(Ухудшение)</font>.
15. Эксперименты с друзьями относительно игры:
    1. Среднее число друзей у игроков <font color="green">(Улучшение)</font>.
    2. Среднее число друзей, играющих в данную игру <font color="green">(Улучшение)</font>.
    3. Среднее число друзей друзей, играющих в данную игру <font color="green">(Улучшение)</font>.
    4. Среднее число друзей друзей у игроков <font color="green">(Улучшение)</font>.
16. One-hot кодирование первых N популярных игр, в которые играл пользователь <font color="green">(Улучшение)</font>.

17. Количество времени, проведенное в топ 100 популярных играх, с учетом переобучения:
    1. Если в train/test нет семпла пользователя с какой-то игрой, ставить 0, если неизвестно – NaN <font color="red">(Ухудшение)</font>.
    2. Если в train/test нет семпла пользователя с какой-то игрой, ставить -1, если неизвестно – NaN, увеличивая информативность признаков <font color="red">(Ухудшение)</font>.
18. Оптимизировать N для пункта 10 <font color="green">(Улучшение)</font>.
19. Число игроков в каждой игре в пределах датасета <font color="green">(Улучшение)</font>.
20. Дополнительный анализ данных о друзьях:
    1. Среднее число друзей у игроков <font color="green">(Улучшение)</font>.
    2. Среднее число друзей, играющих в данную игру <font color="green">(Улучшение)</font>.
    3. Среднее число друзей друзей, играющих в данную игру <font color="green">(Улучшение)</font>.
    4. Среднее число друзей друзей у игроков <font color="green">(Улучшение)</font>.
21. Количество друзей друзей <font color="red">(Ухудшение)</font>.
22. Число игроков, известных из train/test и achievements_stats.csv, играющих в ту же игру:
    1. Друзья <font color="blue">(не дало сильного прироста)</font>.
    2. Друзья друзей <font color="red">(Ухудшение)</font>.
23. Доля жанров для каждого игрока <font color="green">(Улучшение)</font>.
24. Доля жанра для игрока в текущей игре <font color="green">(Улучшение)</font>.
25. Любимый жанр в текущей игре <font color="green">(Улучшение)</font>.
26. Исключить personastate, так как он не информативен без дополнительных характеристик пользователя <font color="red">(Ухудшение)</font>.
27. Работа с рейтингами игр:
    1. Средний рейтинг игр для пользователя <font color="red">(Ухудшение)</font>.
    2. Разница среднего рейтинга игр пользователя и рейтинга игры <font color="red">(Ухудшение)</font>.
28. Число игр, в которые пользователь играл за последние 2 недели <font color="green">(Улучшение)</font>.
29. Число игр, в которые пользователь активно играл (больше 2 часов за последние 2 недели) <font color="red">(Ухудшение)</font>.
30. Среднее время, проведенное в игре за последние 2 недели.
31. Определить оптимальную верхнюю границу времени создания аккаунта и выхода игры, нижняя граница – время игры за последние 2 недели <font color="blue">(не дало сильного прироста)</font>.
32. Использовать user_id как признак, оставив 10% "неизвестными" <font color="red">(Ухудшение)</font>.
33. Использовать CatBoost и оптимизировать параметры на фиксированной валидации/тесте <font color="green">(Улучшение)</font>.
34. Анализ семплов с наибольшей ошибкой (распределение ошибок примерно как нормальное, но выделить общие черты не удалось).

# Код

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import mean_squared_error
from os.path import isdir
import plotly.express as ply
from functools import partial
from os.path import isfile
import networkx as nx
from sklearn.model_selection import KFold
import optuna
from copy import deepcopy
import pickle
import warnings

In [ ]:
RANDOM_STATE = 42
PATH = "./"
SUBMISSION_PATH = "./submission.csv"
PRECOMPUTED_PATH = "./"

train_data = pd.read_csv(f"{PATH}train.csv")
test_data = pd.read_csv(f"{PATH}test.csv")
target = train_data.playtime_forever
train_data.drop(columns='playtime_forever', inplace=True)

target = np.log1p(target)   # Преобразуем для перехода от RMSLE к RMSE
categorical_features = []
bad_features = []           # Признаки, которые удалим   

In [ ]:
def extract_achievement_features(x: pd.Series): 
    '''
    Число достижений, число секретных достижений, лучшее достижение для игрока
    '''
    temp = x.dropna().explode().dropna()
    if temp.shape[0] == 0:
        return (0, 0, 1.0)
    temp = pd.DataFrame(temp.tolist(), columns=['name', 'secret', 'achieved'])
    temp.drop_duplicates(subset=['name'], inplace=True)
    return (temp.shape[0], temp.secret.sum(), temp.achieved.min())

def fetch_neighbors(target, graph=None, depth=1, return_previous_n=True): 
    '''
    Получение соседей неболее чем за depth
    '''
    temp2 = nx.single_source_shortest_path_length(graph, target, cutoff=depth) 
    del temp2[iter(temp2.keys()).__next__()]
    if return_previous_n:
        temp2 = pd.Series(temp2)
        ans = []
        for i in range(1, n):
            ans.append(list(temp2.loc[(temp2 <= i).values].index))
        ans.append(list(temp2.index))
    else:
        ans = list(temp2.keys())
    return ans



if isfile(PRECOMPUTED_PATH + "aggregated.pkl"):  # Есть предпосчитанный файл
    aggregated = pd.read_pickle(PRECOMPUTED_PATH + "aggregated.pkl")
    categorical_features += ['communityvisibilitystate', 'personastate']
else:  
    # aggregated - признаки из train и test, чтобы аггрегировать признаки к ним вместе
    aggregated = pd.concat([train_data[['user_id', 'game_id', 'game_name', 'playtime_2weeks']], test_data[['user_id', 'game_id', 'game_name', 'playtime_2weeks']]])
    aggregated.index = np.arange(aggregated.shape[0])
    
    # выкидываем повторы из achievements_stats.csv и джоиним
    achievement_stats = pd.read_csv(PATH + "achievements_stats.csv",converters={'achievements': lambda x: [] if pd.isnull(x) else eval(x)})
    achievement_stats.drop(columns=['Unnamed: 0'], inplace=True)
    achievement_stats.drop_duplicates(subset=['user_id', 'game_id'], keep='last', inplace=True, ignore_index=True)
    aggregated = aggregated.merge(achievement_stats, how='left', on=['user_id', 'game_id'])
    
    # players_by_game в котором индекс соответсвует game_id, а значение списку игроков
    players_by_game = pd.concat([achievement_stats[['user_id', 'game_id']], aggregated[['user_id', 'game_id']]])
    players_by_game.drop_duplicates(subset=['user_id', 'game_id'], keep='last', inplace=True, ignore_index=True)
    players_by_game = players_by_game.groupby('game_id')['user_id'].agg(lambda x: list(x))
    
    # Обрабатываем достижения через extract_achievement_features
    achievements_aggregated = achievement_stats.groupby('game_id')['achievements'].agg(extract_achievement_features)
    temp = pd.Series([(0, 0, 1.0) for i in range(aggregated.shape[0])])
    mask = aggregated.loc[:, 'game_id'].isin(achievements_aggregated.index).values
    temp.loc[mask] = achievements_aggregated[aggregated.loc[mask, 'game_id'].values].values
    aggregated["achieve_params"] = temp.values
    
    # присоединяем games_details.csv и подстанавливаем первую дату выхода укзананных игр
    game_details = pd.read_csv(PATH + "games_details.csv", index_col=0, converters={'genres': lambda x: eval(x), 'tags': lambda x: eval(x)})
    game_details.loc[game_details.game_id == 602960, 'release_date'] = '2019-06-05'  # Barotrauma
    game_details.loc[game_details.game_id == 1740720, 'release_date'] = '2022-03-08' # Have a Nice Death
    game_details.loc[game_details.game_id == 230410, 'release_date'] = '2013-03-25'  # Warframe
    game_details.loc[game_details.game_id == 1105670, 'release_date'] = '2021-06-03' # The Last Spell
    aggregated = aggregated.merge(game_details, how='left', on=['game_id'])
    
    # присоединяем users.csv
    user_data = pd.read_csv(PATH + "users.csv", index_col=0)
    aggregated = aggregated.merge(temp, how='left', on=['user_id'])
    categorical_features += ['communityvisibilitystate', 'personastate']
    aggregated.release_date = pd.to_datetime(aggregated.release_date).dt.date
    aggregated.timecreated = pd.to_datetime(aggregated.timecreated)
    
    # Представление users_friends.csv в виде графа
    friends_data = pd.read_csv(PATH + "users_friends.csv", index_col=0)
    friends_graph = nx.from_pandas_edgelist(temp, 'user_id', 'friend_id')
    
    # Аггрегация графовых признаков (самое долгое)
    aggregated['max_clique'] = np.nan   # Самая большая клика подграфа игры, в которой состоит игрок
    aggregated['eccentricity'] = np.nan # Эксцентриситет
    connected_components_count = {}     # Число компонент связности в подграфе игры
    clique_number = {}                  # Кликовое число подграфа игры
    for game in aggregated.game_id.unique():  # Идея 31
        users = aggregated.loc[(aggregated.game_id == game).values, 'user_id'].values
        users = sorted(list(set(temp.nodes) & set(users)))
        if len(users) == 0:
            continue
        game_subgraph = nx.induced_subgraph(temp, players_by_game.loc[game])  # Подграф игры
        # Вычисление эксцентриситета
        eccentricity = {}
        connected_components = list(nx.connected_components(game_subgraph))  # Компоненты связности
        connected_components_count[game] = len(connected_components)
        for connected in connected_components:  # Чтобы обойти ошибку с несвязными подграфами, считаем эксцентриситет лишь на компонентах связности
            temp4 = nx.induced_subgraph(game_subgraph, connected)
            eccentricity.update(nx.eccentricity(temp4, sorted(list(set(users) & set(connected)))))
        users1 = aggregated.loc[(aggregated.game_id == game).values, 'user_id']
        aggregated.loc[(aggregated.game_id == game).values, 'eccentricity'] = users1.map(eccentricity)
        
        largest_clique_size = max(len(clique) for clique in nx.find_cliques(game_subgraph))
        
        clique_number[game] = largest_clique_size
        aggregated.loc[(aggregated.game_id == game).values, 'max_clique'] = users1.map(nx.node_clique_number(game_subgraph, nodes=users))
    aggregated['components'] = aggregated.game_id.map(connected_components_count).values
    aggregated['clique_number'] = aggregated.game_id.map(clique_number).values
    
    # Аггрегация друзей 
    user_neighbors = pd.DataFrame({"user_id": aggregated.user_id.unique()})
    temp = user_neighbors.user_id.apply(partial(fetch_neighbors, graph=game_details, n=2))
    user_neighbors['friends'] = temp.str[0]
    user_neighbors['friends_of_friends'] = temp.str[1]
    aggregated = aggregated.merge(user_neighbors, how='left', on=['user_id'])
    aggregated['friends_played_same'] = 0
    aggregated['friends_of_friends_played_same'] = 0
    for game, i in zip(aggregated.game_id.values, np.arange(aggregated.shape[0])):
        temp = pd.Series(aggregated.friends_of_friends.iloc[i]).isin(players_by_game[game]).values
        aggregated.iloc[i, -1] = temp.sum()
        aggregated.iloc[i, -2] = temp[:len(aggregated.friends.iloc[i])].sum()
        
    aggregated.to_pickle(PRECOMPUTED_PATH + "aggregated.pkl")



In [ ]:
GAMES_N_BORDER = 150
TAGS_N_BORDER = 150
precomputed_sets = {'pop': {}, 'var': {}}


def filter_list(x, replace=True, temp=[]):
    """
    Убирает из списка элементы, не содержащиеся в x, если replace=True, то при наличии отличных элементов от
    элементов из temp добавляется строка "__other__"
    """
    if not isinstance(x, list) and pd.isnull(x):
        return []
    flag = False
    ans = []
    for i in x:
        if i in temp:
            ans.append(i)
        else:
            flag = True
    if flag and replace:
        ans.append("__other__")
    return ans

if isfile(PRECOMPUTED_PATH + "games_tags.pkl"):
    with open(PRECOMPUTED_PATH + "games_tags.pkl", 'rb') as f:
        precomputed_sets = pickle.load(f)
else:
    # Временно присоединяем target к aggregated для сбора статистики об играх и тегах
    stats = aggregated.iloc[:train_data.shape[0], :].loc[:, ['game_name', 'tags']].join(target)
    
    # Сбор игр с самой малой дисперсией по target в датасете игр
    games_variance = aggregated.iloc[:train_data.shape[0], :].game_name.value_counts(sort=True)
    games_variance = games_variance[games_variance.values > 50].index
    games_variance = stats.groupby('game_name')['playtime_forever'].agg('std').loc[games_variance.values].sort_values(ascending=True, kind="stable").index[:GAMES_N_BORDER]
    precomputed_sets['var']['list_games'] = games_variance
    
    # Самые популярные игры
    temp = aggregated.game_name.value_counts(sort=True)
    precomputed_sets['pop']['list_games'] = temp.index[:GAMES_N_BORDER]
    
    # Самые популярные теги
    temp = aggregated.tags.explode().value_counts(sort=True)
    temp = temp.index[:TAGS_N_BORDER]
    precomputed_sets['pop']['list_tags'] = temp
    
    # Получение one-hot по объединению тегов по популярности и дисперсии
    temp = aggregated.iloc[:train_data.shape[0], :].tags.explode().value_counts(sort=True)
    temp = temp[temp.values > 50].index
    temp2 = aggregated.iloc[:train_data.shape[0], :].tags.apply(partial(filter_list, replace=False, temp=temp))
    mlb = MultiLabelBinarizer(sparse_output=True)
    temp2 = pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(temp2),
                    columns=mlb.classes_)
    total = {}
    for i in temp2.columns:
        total[i] = target.loc[temp2[i].astype(pd.SparseDtype(bool))].std()
    total = pd.Series(total).sort_values(ascending=True, kind="stable").index[:TAGS_N_BORDER]
    precomputed_sets['var']['list_tags'] = total
    temp = sorted(list(set(precomputed_sets['var']['list_tags']) | set(precomputed_sets['pop']['list_tags'])))
    mlb = MultiLabelBinarizer(sparse_output=False)
    temp2 = aggregated.tags.apply(partial(filter_list, replace=False, temp=temp))
    temp2 = pd.DataFrame(
                    mlb.fit_transform(temp2),
                    columns=[f'tag_{i}' for i in mlb.classes_])
    precomputed_sets['tag_set'] = temp2
    # Сохранение
    with open(PRECOMPUTED_PATH + "games_tags.pkl", 'wb') as f:
        pickle.dump(precomputed_sets, f)

In [ ]:
# Параметры полученные через optuna
N_GAME_POP = 19
N_GAME_VAR = 20
N_TAG_POP = 43
N_TAG_VAR = 8
def contains_other_tag(x, temp=[]):
    """
    Возвращает 1, если есть элемент в x отличный от элементов из temp, иначе 0
    """
    if not isinstance(x, list) and pd.isnull(x):
        return 0
    flag = False
    for i in x:
        if i not in temp:
            flag = True
            break
    return int(flag)

# Получаем списки игр и добавляем признаки по играм/тегам
games_list = sorted(list(set(precomputed_sets['pop']['list_games'][:N_GAME_POP]) | set(precomputed_sets['var']['list_games'][:N_GAME_VAR])))
tags_list = sorted(list(set(precomputed_sets['pop']['list_tags'][:N_TAG_POP]) | set(precomputed_sets['var']['list_tags'][:N_TAG_VAR])))
temp = aggregated.game_name.copy(deep=True)
temp.loc[~(temp.isin(games_list))] = 'Other'
aggregated.loc[:, 'game_name_label'] = LabelEncoder().fit_transform(temp)
feats_list = [f"tag_{i}" for i in tags_list]
categorical_features += feats_list
categorical_features.append("tag___other__")
categorical_features.append('game_name_label')
aggregated = aggregated.join(precomputed_sets['tag_set'].loc[:, feats_list])
aggregated.loc[:, "tag___other__"] = aggregated.tags.apply(partial(contains_other_tag, temp=feats_list))

In [ ]:
def main_achievement(x):  
    '''
    Самое редкое достижение у пользователя 
    '''
    if not isinstance(x, list) and pd.isnull(x):
        return np.nan
    ans = 1.0
    for i in x:
        if ans > i[2]:
            ans = i[2]
    return ans

def count_achievements(x):  
    """
    Количество достижений
    """
    if not isinstance(x, list) and pd.isnull(x):
        return np.nan
    return len(x)


def secret_achievement_ratio(x):  
    '''
    Доля скрытых достижений у пользователя
    '''
    if not isinstance(x, list) and pd.isnull(x):
        return np.nan
    if len(x) == 0:
        return 0.0
    ans = 0
    for i in x:
        ans += int(i[1])
    return ans / len(x)

def count_secret_achievements(x):  
    """
    Количество скрытых достижений у пользователя 
    """
    if not isinstance(x, list) and pd.isnull(x):
        return np.nan
    if len(x) == 0:
        return 0
    ans = 0
    for i in x:
        ans += int(i[1])
    return ans


aggregated['achieve_best'] = aggregated.achievements.apply(main_achievement)
aggregated['achieve_count'] = aggregated.achievements.apply(count_achievements)
aggregated['achieve_hided'] = aggregated.achievements.apply(secret_achievement_ratio)
aggregated['achieve_hided_count'] = aggregated.achievements.apply(count_secret_achievements)
aggregated['achieve_completed'] = aggregated['achieve_count'] / aggregated.achieve_params.str[0]
aggregated['achieve_hided_completed'] = aggregated['achieve_hided_count'] / aggregated.achieve_params.str[1]
aggregated['total_achievements'] = aggregated.achieve_params.str[0]

In [ ]:
def replace_nan_with_list(x):  
    if not isinstance(x, list) and pd.isnull(x):
        return []
    return x

# OHE жанров
aggregated.genres = aggregated.genres.apply(replace_nan_with_list)
mlb = MultiLabelBinarizer(sparse_output=False)
aggregated = aggregated.join(
            pd.DataFrame(
                mlb.fit_transform(aggregated.pop('genres')),
                index=aggregated.index,
                columns=[f"genre_{i}" for i in mlb.classes_]))
categorical_features += [f"genre_{i}" for i in mlb.classes_]
genres_list = [f"genre_{i}" for i in mlb.classes_]

In [ ]:
temp = aggregated.groupby('user_id')
aggregated['games_played'] = temp['game_id'].agg('count')[aggregated.user_id.values].values
aggregated['total_2weeks_playtime'] = temp['playtime_2weeks'].agg('sum')[aggregated.user_id.values].values
aggregated['total_achievements'] = temp['achieve_count'].agg(np.nansum)[aggregated.user_id.values].values 
aggregated['best_achieve_ever'] = temp['achieve_best'].agg(np.nanmin)[aggregated.user_id.values].values 

for feat in genres_list: # Доля жанров игр для игрока
    aggregated[feat + '_played'] = temp[feat].agg("mean")[aggregated.user_id.values].values
aggregated['played_genre_ratio'] = np.max(aggregated.iloc[:, -len(genres_list):-1].values, axis=1)
temp2 = aggregated.iloc[:, -len(genres_list):].values
temp2[~(aggregated.loc[:, genres_list].values.astype(bool))] = -1
aggregated['curr_game_genre_ratio'] = np.max(temp2, axis=1) # Самая большая доля среди жанров
temp2 = np.argmax(temp2, axis=1)
aggregated['curr_game_genre'] = np.array(genres_list)[temp2] # Самый проигрываемый жанр для игрока
temp2 = aggregated.curr_game_genre_ratio == -1
aggregated.loc[temp2.values, 'curr_game_genre'] = 'Unknown' # Если жанры неизвестны
aggregated.loc[:, 'curr_game_genre_labeled'] = LabelEncoder().fit_transform(aggregated.loc[:, 'curr_game_genre'].values) # Числовые значения для curr_game_genre
categorical_features.append('curr_game_genre_labeled')
aggregated['mean_rating'] = temp['rating'].agg(np.nanmean)[aggregated.user_id.values].values 
aggregated['rating_normed'] = aggregated.rating.values - aggregated.mean_rating.values # Разница рейтинга игры и прошлого признака

aggregated['games_played_in_2weeks'] = temp['playtime_2weeks'].agg(lambda x: (x > 0).sum())[aggregated.user_id.values].values
aggregated['games_actively_played_in_2weeks'] = temp['playtime_2weeks'].agg(lambda x: (x > 120).sum())[aggregated.user_id.values].values
bad_features += ['played_genre_ratio', 'mean_rating', 'rating_normed', 'games_actively_played_in_2weeks']

In [ ]:
temp = aggregated.groupby('game_id')
aggregated['mean_playtime_2weeks'] = temp['playtime_2weeks'].agg(np.nanmean)[aggregated.game_id.values].values
aggregated['players_count'] = temp['user_id'].agg('count')[aggregated.game_id.values].values
moment = pd.Series([pd.to_datetime('2023-03-09')] * aggregated.shape[0])
aggregated['release_date'] = pd.to_datetime(aggregated['release_date'])
aggregated['release_diff'] = (moment - aggregated['release_date']).dt.days
aggregated['account_time'] = (moment - aggregated['timecreated']).dt.total_seconds() / (60 * 60 * 24)
aggregated['friends_count'] = aggregated.friends.apply(lambda x: 0 if (not isinstance(x, list) and pd.isnull(x)) else len(x))
aggregated['friends_of_friends_count'] = aggregated.friends_of_friends.apply(lambda x: 0 if (not isinstance(x, list) and pd.isnull(x)) else len(x))
# Среднее число друзей игроков из игры
aggregated['mean_friends'] = temp['friends_count'].agg(np.nanmean)[aggregated.game_id.values].values
# Среднее число друзей игроков из игры, которые играют в эту же игру
aggregated['mean_friends_played_same'] = temp['friends_played_same'].agg(np.nanmean)[aggregated.game_id.values].values
# Среднее число друзей друзей игроков из игры, которые играют в эту же игру
aggregated['mean_friends_of_friends_played_same'] = temp['friends_of_friends_played_same'].agg(np.nanmean)[aggregated.game_id.values].values
# Среднее число друзей игроков из игры, которые играют в эту же игру
aggregated['mean_friends_of_friends'] = temp['friends_of_friends_count'].agg(np.nanmean)[aggregated.game_id.values].values

# Отношение максимальной клики для игрока к кликовому числу подграфа игры
aggregated['max_clique_norm'] = aggregated.max_clique.values / aggregated.clique_number.values

In [ ]:
# One-hot самых популярных игр, в которые играл пользователь
temp = aggregated.groupby("user_id")["game_name"]\
                 .agg(lambda x: list(x.unique()))\
                 .apply(partial(filter_list, replace=False, temp=precomputed_sets['pop']['list_games'][:100]))
temp = pd.DataFrame({"user_id": temp.index, "games": temp.values})
mlb = MultiLabelBinarizer(sparse_output=False)
temp = temp.join(pd.DataFrame(
                mlb.fit_transform(temp.games),
                columns=[f'game_{i}_played' for i in mlb.classes_]))
categorical_features += [f'game_{i}_played' for i in mlb.classes_]
temp.drop(columns=["games"], inplace=True)
aggregated = aggregated.merge(temp, how="left", on="user_id")


In [ ]:
SEEK_TOP_GAMES = 100
top_games = aggregated.game_id.value_counts(sort=True, ascending=False).index[:SEEK_TOP_GAMES]
y_extended = pd.Series(data=-2.0, index=np.arange(aggregated.shape[0]))
y_extended.iloc[:train_data.shape[0]] = target.values
aggregated["y_ext"] = y_extended.values

temp = aggregated.groupby(["user_id", "game_id"])["y_ext"].agg('sum')
top_games = pd.Series(top_games)
arr = []
for user in aggregated.user_id.unique():
    arr.append(top_games.map(temp[user]).values)
arr = np.vstack(arr)
arr[np.isnan(arr)] = -1.0
arr[arr == -2] = np.nan
arr = pd.DataFrame(arr, columns=[f"time_{i}_played" for i in top_games.values])
arr["user_id"] = aggregated.user_id.unique()
aggregated = aggregated.merge(arr, how="left", on="user_id")

indexes_games = pd.Series(np.arange(SEEK_TOP_GAMES), index=top_games.values)
for i in range(train_data.shape[0]):
    if aggregated.game_id.iloc[i] in top_games.values:
        aggregated.iloc[i, -100 + indexes_games.loc[aggregated.game_id.iloc[i]]] = np.nan
aggregated.drop(columns=["y_ext"], inplace=True)

bad_features += [f"time_{i}_played" for i in top_games.values]

In [ ]:
UNREDACTED_AGGREGATED = aggregated.copy(deep=True)
UNREDACTED_TRAIN = aggregated.iloc[:train_data.shape[0]].copy(deep=True)
def get_data(X_on: pd.DataFrame =None, y_on: pd.DataFrame=None, mas=None, apply_to=[], random_state=None, gamma=0.55):
    """
    Создаем признаки для классификации на основе y_on, используя X_on.
    
    mas: маска для выбора признаков, чтобы не обучаться на всех признаках сразу.
    apply_to: наборы данных, к которым мы добавляем новые признаки и возвращаем их после обработки.
    random_state: параметр для управления случайностью.
    gamma: доля y, известная для агрегации признаков, чтобы модель могла обучаться в условиях, когда некоторые значения отсутствуют.
    """
    rand = np.random.RandomState(random_state)

    if X_on is not None and y_on is not None:
        X_ondate = X_on
        y_ondate = y_on
    else:
        X_ondate = UNREDACTED_TRAIN.loc[mas]
        y_ondate = target.loc[mas]
    X_ondate['y'] = y_ondate
    mask_gamma = rand.uniform(size=[X_ondate.shape[0]]) < gamma
    
    ans = []
    new_feats = []
    new_feats_names = []
    grouped_by = []
    
    temp = X_ondate.loc[(X_ondate.tag_Singleplayer == 1) & mask_gamma].groupby('user_id')['y'].agg('mean')
    new_feats.append(temp)
    new_feats_names.append('singleplayer_played_mean')
    grouped_by.append('user_id')
    
    temp = X_ondate.loc[(X_ondate.tag_Multiplayer == 1) & mask_gamma].groupby('user_id')['y'].agg('mean')
    new_feats.append(temp)
    new_feats_names.append('multiplayer_played_mean')
    grouped_by.append('user_id')
    
    temp = X_ondate.loc[mask_gamma].groupby('user_id')['y'].agg('mean')
    new_feats.append(temp)
    new_feats_names.append('played_mean')
    grouped_by.append('user_id')
    
    temp = X_ondate.loc[mask_gamma].groupby('game_id')['y'].agg('mean')
    new_feats.append(temp)
    new_feats_names.append('games_played_mean')
    grouped_by.append('game_id')
    
    temp = X_ondate.loc[mask_gamma].groupby('game_name_label')['y'].agg(lambda x: np.nanmean(x > np.log1p(120)))
    new_feats.append(temp)
    new_feats_names.append('game_played_more_2h')
    grouped_by.append('game_name_label')
    
    transformer = X_ondate.groupby('game_id')['y'].agg('mean')
    temp = pd.Series(np.nan, index=np.arange(X_ondate.shape[0]))
    mask = X_ondate.loc[:, 'game_id'].isin(transformer.index).values
    temp.loc[mask] = transformer[X_ondate.loc[mask, 'game_id'].values].values
    y_temp = pd.Series(np.nan, index=np.arange(X_ondate.shape[0]))
    y_temp.loc[mask_gamma] = (X_ondate.y.values - temp.values)[mask_gamma]
    temp = pd.DataFrame({'user_id': X_ondate.user_id, 'y': y_temp.values})
    temp = temp.groupby('user_id')['y'].agg('mean')
    new_feats.append(temp)
    new_feats_names.append('activity_on_game')
    grouped_by.append('user_id')

    
    X_ondate.drop(columns=['y'], inplace=True)
    apply_list = [X_ondate] # Список всех датасетов, к которым применяется преобразование
    apply_list += apply_to
    for X_dataset in apply_list:
        for name, transformer, grouped_on in zip(new_feats_names, new_feats, grouped_by):
            temp = pd.Series(np.nan, index=np.arange(X_dataset.shape[0]))
            mask = X_dataset.loc[:, grouped_on].isin(transformer.index).values
            temp.loc[mask] = transformer[X_dataset.loc[mask, grouped_on].values].values
            X_dataset[name] = temp.values
        temp = pd.Series(np.nan, index=np.arange(X_dataset.shape[0]))
        mask = (X_dataset.tag_Singleplayer == 1).values
        temp.loc[mask] = X_dataset.loc[mask, 'singleplayer_played_mean']
        mask = (X_dataset.tag_Multiplayer == 1).values
        temp.loc[mask] = X_dataset.loc[mask, 'multiplayer_played_mean']
        X_dataset['played_on_tag'] = temp.values
        X_dataset['activity_game_played_mean'] = X_dataset.games_played_mean + X_dataset.activity_on_game
        ans.append(X_dataset)
    return tuple(ans)
bad_features.append('game_played_more_2h')

In [ ]:
def pred_trunc(X_test, y_test: pd.Series):
    """
        Функция, которая подгоняет y под временные рамки признаков:
        1. Если y < np.log1p(playtime_2weeks), то у = np.log1p(playtime_2weeks)
        2. Если y > np.log1p(release_diff * 24 * 60 * C), то y = np.log1p(release_diff * 24 * 60 * C), C = 0.3
        3. Если y > np.log1p(account_time * 24 * 60 * C), то y = np.log1p(account_time * 24 * 60 * C), C = 0.1
    """
    upper = pd.Series(np.inf, index=X_test.index)
    temp = np.log1p(X_test.account_time.copy() * 24 * 60 * 0.1)
    temp.loc[temp.isna().values] = np.inf
    upper.loc[upper.values > temp.values] = temp.loc[upper > temp].values
    temp = X_test.release_diff.copy()
    temp.loc[temp.values < 0] = 0
    temp = np.log1p(temp.values * 24 * 60 * 0.3)
    temp[np.isnan(temp)] = np.inf
    upper.loc[upper.values > temp] = temp[upper.values > temp]
    y_test.loc[y_test.values > upper.values] = upper.loc[y_test.values > upper.values].values
    temp = np.log1p(X_test.playtime_2weeks)
    y_test.loc[y_test.values < temp.values] = temp[y_test.values < temp.values].values
    return y_test

In [ ]:
bad_features += ["game_id", "user_id", "profilestate", 'achieve_count', 'achieve_hided_count', 'friends_of_friends_played_same', 'friends_of_friends_count', 'clique_number'] + list(aggregated.columns[(aggregated.dtypes == object) | (aggregated.dtypes == 'datetime64[ns]')])
bad_features = sorted(list(set(bad_features) & set(aggregated.columns)))
categorical_all = deepcopy(categorical_features)
categorical_features = sorted(list(set(categorical_features) - set(bad_features)))

Обучение

In [ ]:
train_set_aggr = aggregated.iloc[:train_data.shape[0], :].copy()
test_set_aggr = aggregated.iloc[train_data.shape[0]:, :].copy()
CAT_PARAMS={'max_depth': 12,
            'max_leaves': 164,
            'min_child_samples': 34,
            'reg_lambda': 3.977768225983847,
            'grow_policy': 'Lossguide',
            'n_estimators': 5000}

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_set_aggr, target, test_size=0.05, random_state=RANDOM_STATE)
X_train, X_val, test_set_aggr = get_data(X_train, y_train, apply_to=[X_val, test_set_aggr], random_state=RANDOM_STATE)
X_train.drop(columns=bad_features, inplace=True)
X_val.drop(columns=bad_features, inplace=True)
test_set_aggr.drop(columns=bad_features, inplace=True)


In [ ]:
model = CatBoostRegressor(random_state=RANDOM_STATE, **CAT_PARAMS)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', category=FutureWarning)
    model.fit(X_train, y_train,
              eval_set=(X_val, y_val),
              early_stopping_rounds=100,
              cat_features=categorical_features,
              use_best_model=True,
              metric_period=20)


In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', category=FutureWarning)
    preds = model.predict(test_set_aggr)
preds = pred_trunc(test_set_aggr, pd.Series(preds))
preds = np.expm1(preds)
submission = pd.DataFrame({"index": test_data["index"],
                           "playtime_forever": preds})
submission.to_csv(SUBMISSION_PATH, index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
answers = []
TRIES = 10
for tries in range(TRIES):
    X_train, X_test, y_train, y_test = train_test_split(UNREDACTED_AGGREGATED.iloc[:train_data.shape[0]], target, test_size=0.1, random_state=tries)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=tries)
    X_train, X_val, X_test = get_data(X_train, y_train, apply_to=[X_val, X_test], gamma=0.58, random_state=tries)
    X_train.drop(columns=bad_features, inplace=True)
    X_val.drop(columns=bad_features, inplace=True)
    X_test.drop(columns=bad_features, inplace=True)
    model = CatBoostRegressor(verbose=-1, random_state=tries, **CAT_PARAMS)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=FutureWarning)
        model.fit(X_train, y_train,
                  eval_set=(X_val, y_val),
                  early_stopping_rounds=60,
                  cat_features=categorical_features,
                  use_best_model=True,
                  verbose=False)
        temp = model.predict(X_test)
    temp = pred_trunc(X_test, pd.Series(temp))
    temp2 = mean_squared_error(y_test, temp, squared=False)
    answers.append(temp2)
    print(f"Current RMSE: {temp2}, mean: {np.mean(answers)}")
answers = np.array(answers)
var = np.std(answers)
answers.sort()
print(f"Error = {np.mean(answers[1:-1])}, std = {var}, mean std = {var / np.sqrt(TRIES)}")

Оптимизация

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)
X_train, X_test, y_train, y_test = train_test_split(UNREDACTED_AGGREGATED.iloc[:train_data.shape[0]], target, test_size=0.1, random_state=42)
# X_train, X_test = get_data(X_train, y_train, apply_to=[X_test])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train, X_val, X_test = get_data(X_train, y_train, apply_to=[X_val, X_test], gamma=0.58, random_state=42)
X_train.drop(columns=bad_features, inplace=True)
X_val.drop(columns=bad_features, inplace=True)
X_test.drop(columns=bad_features, inplace=True)
def func(trial: optuna.Trial):
    sample_params = {
        'n_estimators': 1000000,
        'max_depth': trial.suggest_int("max_depth", 7, 15),
        'max_leaves': trial.suggest_int("max_leaves", 5, 200, log=True),
        'min_child_samples': trial.suggest_int("min_child_samples", 1, 300, log=True),
        'reg_lambda': trial.suggest_float("reg_lambda", 1e-3, 100.0, log=True),
        'grow_policy': 'Lossguide',
        'verbose': -1
    }
    model = CatBoostRegressor(**sample_params)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=FutureWarning)
        model.fit(X_train, y_train,
                  eval_set=(X_val, y_val),
                  early_stopping_rounds=100,
                  cat_features=categorical_features,
                  use_best_model=True,
                  verbose=False)
        temp = model.predict(X_test)
    temp = pred_trunc(X_test, pd.Series(temp))
    temp2 = mean_squared_error(y_test, temp, squared=False)
    return temp2

In [ ]:
opt = optuna.create_study(sampler=optuna.samplers.TPESampler(n_startup_trials=100), direction='minimize')

In [ ]:
opt.optimize(func, n_trials=150)

In [ ]:
opt.best_params

In [ ]:
optuna.visualization.plot_slice(opt)